In [9]:
%load_ext autoreload
%autoreload 2

from dataclasses import dataclass
from querychains import with_context, Context, OpenAiChatEngine, start_server, AnthropicEngine, QueryEngine
import json

def show(obj):
    print(json.dumps(obj, indent=2))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Minimal context
with Context("MyContext") as c:
    pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uuid": "efcfaaa8-ee90-4b96-b633-0c05b71ff035"
}


In [11]:
# Context nesting
with Context("MyContext") as c:
    with Context("ChildContext1"):
        pass
    with Context("ChildContext2"):
        pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uuid": "39826402-9404-4f19-8cf8-1af866212349",
  "children": [
    {
      "_type": "Context",
      "name": "ChildContext1",
      "uuid": "9a1958ee-1c2f-46d6-80ba-aa999fd9143b"
    },
    {
      "_type": "Context",
      "name": "ChildContext2",
      "uuid": "eebfddae-9cc6-4e9a-b9e9-b2e3ccf791bc"
    }
  ]
}


In [14]:
# Inputs and outputs
with Context("MyContext", inputs={"a": 10, "b": 20}) as c:
    c.set_result(30)

show(c.to_dict())

{
  "_type": "Context",
  "name": "MyContext",
  "uuid": "770a953b-ec6a-40a7-9640-6921367618fb",
  "inputs": {
    "a": 10,
    "b": 20
  },
  "result": 30
}


In [15]:
# Context decorator
@with_context
def my_function(a, b):
    return a + b

with Context("root") as c:
    my_function(10, 20)
    
show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "c60b9b38-c8b7-41cd-a18c-e1a229c19c39",
  "children": [
    {
      "_type": "Context",
      "name": "my_function",
      "uuid": "379b8893-67c3-4d53-a346-588164114681",
      "kind": "call",
      "inputs": {
        "a": 10,
        "b": 20
      },
      "result": 30
    }
  ]
}


In [12]:
# Error handling

@with_context
def my_function(a, b):
    raise Exception("Oops")

try:
    with Context("root") as c:
        my_function(10, 20)
except:
    pass

show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "e50b7f76-69d3-4504-af4f-be695cc4aa85",
  "state": "error",
  "error": {
    "_type": "error",
    "name": "Oops"
  },
  "children": [
    {
      "_type": "Context",
      "name": "my_function",
      "uuid": "111e2791-68ba-4665-87b3-98e1722fb3f0",
      "state": "error",
      "kind": "call",
      "inputs": {
        "a": 10,
        "b": 20
      },
      "error": {
        "_type": "error",
        "name": "Oops"
      }
    }
  ]
}


In [13]:
# Dataclasses

@dataclass
class Person:
    name: str
    age: int
    
@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "c1d83bae-b31b-40a3-9c97-2f77dff566cc",
  "children": [
    {
      "_type": "Context",
      "name": "say_hi",
      "uuid": "d1e0c4a4-b28c-485e-9fa0-012155ae0eec",
      "kind": "call",
      "inputs": {
        "person": {
          "name": "Alice",
          "age": 21,
          "_type": "Person"
        }
      },
      "result": "Hi Alice!"
    }
  ]
}


In [14]:
# Data browser

server = start_server()  # Writes an URL where the server is running

INFO:     Started server process [43702]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:39709 (Press CTRL+C to quit)


INFO:     127.0.0.1:60282 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:60282 - "GET /static/js/main.2ae1ca2b.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:60294 - "GET /static/css/main.073c9b0a.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:60294 - "GET /logo192.png HTTP/1.1" 200 OK
INFO:     127.0.0.1:60282 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     127.0.0.1:60296 - "GET /contexts/list HTTP/1.1" 200 OK


In [21]:
@dataclass
class Person:
    name: str
    age: int
    
@with_context
def say_hi(person):
    return f"Hi {person.name}!"

with Context("root") as c:
    person = Person("Alice", 21)
    say_hi(person)

server.add_context(c)

INFO:     127.0.0.1:55156 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:55156 - "GET /contexts/list HTTP/1.1" 200 OK
INFO:     127.0.0.1:55156 - "GET /contexts/uuid/20ea4dd9-d87e-461e-ab17-830aba97c060 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55156 - "GET /contexts/uuid/b8edbc0f-27dd-4e59-89a6-65f4740bda71 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55156 - "GET /contexts/uuid/5a2e80f0-0b31-4462-b89f-861802cbbeff HTTP/1.1" 200 OK


In [3]:
# LLM queries

engine = OpenAiEngine()

with Context("root") as c:
        response = engine.query("Hi are you?")
        engine.query("Is the following text genereted by LLM?\n\n" + response)
show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "9c4b98d3-e099-4031-808b-90ab938b2792",
  "children": [
    {
      "_type": "Context",
      "name": "OpenAiChat gpt-3.5-turbo",
      "uuid": "d5e79f60-e6b9-4ef3-95c8-39b705c18734",
      "kind": "query",
      "inputs": {
        "prompt": "Hi are you?",
        "conf": {
          "api": "OpenAiChat",
          "model": "gpt-3.5-turbo",
          "temperature": 0.7,
          "max_tokens": 1024,
          "_type": "QueryConf"
        }
      },
      "result": "As an AI language model, I don't have emotions, but I'm functioning well. How can I assist you today?"
    },
    {
      "_type": "Context",
      "name": "OpenAiChat gpt-3.5-turbo",
      "uuid": "53c008a5-5149-4768-a4fd-65150bff1d9a",
      "kind": "query",
      "inputs": {
        "prompt": "Is the following text genereted by LLM?\n\nAs an AI language model, I don't have emotions, but I'm functioning well. How can I assist you today?",
        "conf": {
          "api"

In [8]:
# async LLM queries
import asyncio 

engine1 = OpenAiChatEngine()
engine2 = AnthropicEngine()

@with_context
async def make_queries(engine: QueryEngine):
    response = await engine.aquery("Hi are you?")
    return await engine.aquery("Is this nice response?\n\n" + response)  

with Context("root") as c:
    q1 = make_queries(engine1)
    q2 = make_queries(engine2)

    await q1
    await q2

show(c.to_dict())

{
  "_type": "Context",
  "name": "root",
  "uuid": "93baaec7-67eb-4b6c-8a29-2df8f3bad052",
  "children": [
    {
      "_type": "Context",
      "name": "make_queries",
      "uuid": "a1a3e0ad-9784-4c46-8023-714764694d47",
      "kind": "acall",
      "inputs": {
        "engine": {
          "model": "gpt-3.5-turbo",
          "temperature": 0.7,
          "_type": "OpenAiChatEngine"
        }
      },
      "result": "Yes, this is a polite and helpful response.",
      "children": [
        {
          "_type": "Context",
          "name": "OpenAiChat gpt-3.5-turbo",
          "uuid": "6ea7d64f-fe9a-4890-bd78-f3703d3f67eb",
          "kind": "query",
          "inputs": {
            "prompt": "Hi are you?",
            "conf": {
              "api": "OpenAiChat",
              "model": "gpt-3.5-turbo",
              "temperature": 0.7,
              "max_tokens": 1024,
              "_type": "QueryConf"
            }
          },
          "result": "As an AI language model, I don'